In [1]:
!pip install openai sqlite-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.5 MB/s eta 0:00:00


In [2]:
import sqlite3
from datetime import datetime


In [3]:
# Connect to SQLite (will create a file named 'agent_memory.db')
conn = sqlite3.connect("agent_memory.db")
cursor = conn.cursor()

# Create a table for session memory
cursor.execute('''
CREATE TABLE IF NOT EXISTS memory (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    role TEXT,
    message TEXT,
    timestamp TEXT
)
''')
conn.commit()


In [4]:
def save_message(session_id, role, message):
    cursor.execute('''
    INSERT INTO memory (session_id, role, message, timestamp)
    VALUES (?, ?, ?, ?)
    ''', (session_id, role, message, datetime.now().isoformat()))
    conn.commit()

def get_session_memory(session_id):
    cursor.execute('SELECT role, message FROM memory WHERE session_id=?', (session_id,))
    return cursor.fetchall()

def clear_session_memory(session_id):
    cursor.execute('DELETE FROM memory WHERE session_id=?', (session_id,))
    conn.commit()


In [5]:
def chat_with_agent(session_id, user_input):
    if user_input.lower() == "/reset":
        clear_session_memory(session_id)
        return "🔄 Memory cleared for this session!"

    # Retrieve past messages
    memory = get_session_memory(session_id)

    # Build simple context
    context = "Conversation so far:\n"
    for role, msg in memory[-3:]:  # last 3 messages for short memory
        context += f"{role}: {msg}\n"

    # Simple rule-based agent
    if "name" in user_input.lower():
        reply = "I'm your memory agent 🤖. I can remember what you tell me!"
    elif "remember" in user_input.lower():
        reply = "Got it! I'll remember that. 🧠"
    elif "favorite color" in user_input.lower():
        reply = "You told me your favorite color is blue earlier!"
    else:
        reply = "Thanks! I’ve noted that."

    # Save both user and agent messages
    save_message(session_id, "User", user_input)
    save_message(session_id, "Agent", reply)

    return reply


In [7]:
def chat_with_agent(session_id, user_input):
    user_input = user_input.lower()

    if user_input == "/reset":
        clear_session_memory(session_id)
        return "🔄 Memory cleared for this session!"

    # Retrieve memory for this session
    memory = get_session_memory(session_id)

    # Store user message
    save_message(session_id, "User", user_input)

    # --- Intelligent memory storage ---
    reply = None
    if "my name is" in user_input:
        name = user_input.split("my name is")[-1].strip().capitalize()
        save_message(session_id, "System", f"name:{name}")
        reply = f"Nice to meet you, {name}! I'll remember your name. 😊"

    elif "my age is" in user_input:
        age = user_input.split("my age is")[-1].strip()
        save_message(session_id, "System", f"age:{age}")
        reply = f"Got it! You're {age} years old. 🎂"

    elif "i am studying" in user_input:
        study = user_input.split("i am studying")[-1].strip()
        save_message(session_id, "System", f"study:{study}")
        reply = f"Cool! You're studying {study}. I'll note that down. 🎓"

    # --- Intelligent recall ---
    elif "what is my name" in user_input:
        cursor.execute("SELECT message FROM memory WHERE session_id=? AND message LIKE 'name:%'", (session_id,))
        result = cursor.fetchall()
        if result:
            name = result[-1][0].split(":")[1]
            reply = f"Your name is {name}. 😊"
        else:
            reply = "I don't know your name yet. Tell me using 'My name is ...'."

    elif "what is my age" in user_input:
        cursor.execute("SELECT message FROM memory WHERE session_id=? AND message LIKE 'age:%'", (session_id,))
        result = cursor.fetchall()
        if result:
            age = result[-1][0].split(":")[1]
            reply = f"You're {age} years old. 🎂"
        else:
            reply = "You haven’t told me your age yet."

    elif "what am i studying" in user_input:
        cursor.execute("SELECT message FROM memory WHERE session_id=? AND message LIKE 'study:%'", (session_id,))
        result = cursor.fetchall()
        if result:
            study = result[-1][0].split(":")[1]
            reply = f"You're studying {study}. 📚"
        else:
            reply = "You haven't told me that yet."

    # --- Default ---
    if not reply:
        reply = "Got it! I’ll remember that. 🧠"

    # Save agent reply
    save_message(session_id, "Agent", reply)
    return reply


In [ ]:
session_id = "session_001"

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("👋 Goodbye!")
        break
    response = chat_with_agent(session_id, user_input)
    print("Agent:", response)


You: My name is Sreeja
Agent: Nice to meet you, Sreeja! I'll remember your name. 😊
You: My age is 20
Agent: Got it! You're 20 years old. 🎂
You: My place is kakinada
Agent: Got it! I’ll remember that. 🧠
You: what is my name
Agent: Your name is Sreeja. 😊
You: what is my age
Agent: You're 20 years old. 🎂
You: what is my plase
Agent: Got it! I’ll remember that. 🧠
You: what is my place
Agent: Got it! I’ll remember that. 🧠
